In [190]:
# Apprentissage
# Mettre _ pour commencer une fonction indique qu'elle est privée
#Je peux pas utiliser pad car

In [208]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [209]:
import sys
sys.path.append('/content/drive/MyDrive/Projet Urchadee/Cross encoder :multilabel classification')

In [44]:
from transformers import BertTokenizer

# Charger le tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # ou 'bert-base-multilingual-cased' pour le multilingue

# Définir vos labels et le texte
labels = ["label 1", "label 2", "label 3"]  # Liste de vos labels
text = "This is an example of text."       # Texte principal

# Transformation en une seule chaîne avec "[CLS]" avant chaque élément
labels_concatenated = " ".join(f"{tokenizer.cls_token} {label}" for label in labels)
input = " ".join([labels_concatenated, f"{tokenizer.sep_token} {text} {tokenizer.sep_token}"])
encoding = tokenizer(input, return_tensors='pt', add_special_tokens=False)


input_ids = encoding['input_ids']
attention_mask = encoding['attention_mask']

all_cls_positions = (input_ids == tokenizer.cls_token_id).nonzero(as_tuple=True)[1]
label_cls_positions = all_cls_positions[:len(labels)]

In [45]:
label_cls_positions

tensor([0, 3, 6])

In [202]:
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from torch.nn.utils.rnn import pad_sequence

class CrossEncoderModel(nn.Module):
    def __init__(self, model_name, max_num_labels):
        """
        Args:
            model_name (str): Name of the pretrained model (e.g., "bert-base-uncased").
            max_num_labels (int): Maximum number of labels that any text can have.
        """
        super(CrossEncoderModel, self).__init__()
        # TODO: 1) Load a pretrained transformer model and its tokenizer.
        #       2) Initialize a classifier head (nn.Linear) that maps from hidden_size -> 1 (for each label).

        # Example:
        self.encoder = AutoModel.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.classifier = torch.nn.Linear(self.encoder.config.hidden_size, 1)

        self.max_num_labels = max_num_labels

    def _build_crossencoder_inputs(self, text, labels):
        """
        Build a single cross-encoder input for one example:
            [CLS] label_1 [CLS] label_2 ... [CLS] label_k [SEP] text [SEP]

        Returns:
            input_ids (torch.LongTensor): Token indices for the concatenated sequence.
            attention_mask (torch.LongTensor): Attention mask for that sequence.
            label_cls_positions (torch.LongTensor): Indices of the `[CLS]` tokens corresponding to each label.
        """
        # TODO: 1) Construct a string that inserts "[CLS] " before each label,
        #          then "[SEP]" + text + "[SEP]" at the end.
        #       2) Tokenize this string without automatically adding special tokens,
        #          because you are adding them manually.
        #       3) Find the positions where `token_id == self.tokenizer.cls_token_id`
        #          for the label embeddings.
        #       4) Return input_ids, attention_mask, and label_cls_positions.

        # Pseudocode:
        # label_part = ""
        # for label in labels:
        #     label_part += f"{self.tokenizer.cls_token} {label} "
        #
        # combined_str = f"{label_part}{self.tokenizer.sep_token} {text} {self.tokenizer.sep_token}"
        #
        # encoding = self.tokenizer(
        #     combined_str,
        #     add_special_tokens=False,
        #     return_tensors="pt"
        # )
        #
        # input_ids = ...
        # attention_mask = ...
        #
        # all_cls_positions = (input_ids == self.tokenizer.cls_token_id).nonzero(as_tuple=True)[0]
        # label_cls_positions = all_cls_positions[:len(labels)]
        #
        # return input_ids, attention_mask, label_cls_positions

        labels_concatenated = " ".join(f"{self.tokenizer.cls_token} {label}" for label in labels)
        combined_str = " ".join([labels_concatenated, f"{self.tokenizer.sep_token} {text} {self.tokenizer.sep_token}"])
        encoding = tokenizer(combined_str, add_special_tokens=False, padding=True, return_tensors='pt')

        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']

        all_cls_positions = (input_ids == self.tokenizer.cls_token_id).nonzero(as_tuple=True)[1]
        label_cls_positions = all_cls_positions[:len(labels)]

        return input_ids, attention_mask, label_cls_positions

    def forward(self, texts, batch_labels):
        """
        Args:
            texts (List[str]): List of texts with batch size B.
            batch_labels (List[List[str]]): List of label-lists for each text.

        Returns:
            scores (torch.FloatTensor): Shape [B, max_num_labels],
                                        containing the predicted relevance for each label.
            mask (torch.BoolTensor): Shape [B, max_num_labels],
                                     indicating which label positions are valid for each example.
        """
        # TODO: 1) For each example (text + label list), build crossencoder inputs using `_build_crossencoder_inputs`.
        #       2) Use the tokenizer's `.pad(...)` to handle variable-length sequences across the batch.
        #       3) Pass the padded batch through your transformer encoder.
        #       4) Gather the [CLS] embeddings for each label from the `last_hidden_state`.
        #       5) Pad them to [B, max_num_labels, hidden_dim].
        #       6) Pass through a classifier (linear layer) => [B, max_num_labels].
        #       7) Apply sigmoid if doing multi-label classification.
        #       8) Return (scores, mask).

        # Step-by-step outline:
        # 1. Prepare lists for input_ids, attention_masks, label_positions_batch.
        # 2. For i in range(len(texts)):
        #       input_ids_i, att_mask_i, label_pos_i = self._build_crossencoder_inputs(texts[i], batch_labels[i])
        #       store them in lists
        #
        # 3. Use tokenizer.pad(...) to get padded input_ids and attention_mask for the entire batch.
        #
        # 4. self.encoder(...) => get last_hidden_state (B, seq_len, hidden_dim).
        #
        # 5. For each example i, gather the embeddings from the label positions.
        # 6. Construct a zero tensor padded_label_embeddings (B, max_num_labels, hidden_dim).
        #    Construct a boolean mask (B, max_num_labels) to mark real vs. padded labels.
        #
        # 7. Run classifier => logits => apply sigmoid => scores.
        #
        # return (scores, mask)




        # # Conversion en listes
        # dict_return = {"input_ids" : list(liste1), "attention_mask" :  list(liste2)}#, "label_cls_positions" : list(liste3)}

        #return dict_return
        B = len(texts)
        max_num_labels = self.max_num_labels
        hidden_dim = self.encoder.config.hidden_size


        input_ids, att_mask, label_pos = [], [], []
        for i in range(B):
            input_ids_i, att_mask_i, label_pos_i = self._build_crossencoder_inputs(texts[i], batch_labels[i])

            input_ids.append(input_ids_i.squeeze(0))
            att_mask.append(att_mask_i.squeeze(0))
            label_pos.append(label_pos_i.squeeze(0))

        inputs = tokenizer.pad({"input_ids" : input_ids, "attention_mask" : att_mask}, padding=True)
        outputs = self.encoder(**inputs)

        cls_embeddings = torch.zeros(B, max_num_labels, hidden_dim)
        last_hidden_state = outputs.last_hidden_state
        mask_labels = torch.zeros(B, max_num_labels, dtype=torch.bool)
        for i in range(B):
            count = label_pos[i].shape[0]
            if count>0:
                cls_embeddings[i , :count, :] = last_hidden_state[i, label_pos[i].tolist(), :]
                mask_labels[i, :count] = True


        scores = self.classifier(cls_embeddings)
        scores = torch.sigmoid(scores)

        return scores, mask_labels

    @torch.no_grad()
    def forward_predict(self, texts, labels):
        """
        Args:
            texts (List[str]): List of input texts.
            labels (List[List[str]]): List of labels corresponding to each text.

        Returns:
            A list of dictionaries, each containing:
               {
                   "text": str,
                   "scores": { label: float_score, ... }
               }
        """

        scores, masks = self.forward(texts, labels)
        results = []
        for i, text in enumerate(texts):
            text_result = {}
            for j, label in enumerate(labels[i]):
                if masks[i, j]:
                    text_result[label] = float(f"{scores[i, j].item():.2f}")
            results.append({"text": text, "scores": text_result})
        return results

In [205]:
if __name__ == "__main__":
    model_name = "bert-base-uncased"
    max_num_labels = 5

    model = CrossEncoderModel(model_name, max_num_labels)

    texts = ["I love machine learning.", "Deep learning models are powerful."]
    batch_labels = [
        ["AI", "Machine Learning"],
        ["Deep Learning", "Neural Networks", "AI"]
    ]
    a = model.forward_predict(texts, batch_labels)

In [206]:
a

[{'text': 'I love machine learning.',
  'scores': {'AI': 0.42, 'Machine Learning': 0.43}},
 {'text': 'Deep learning models are powerful.',
  'scores': {'Deep Learning': 0.49, 'Neural Networks': 0.5, 'AI': 0.51}}]